<a href="https://colab.research.google.com/github/noname1711/cryptography_theory/blob/main/hill_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from google.colab import files

In [2]:
# Chuyển ký tự sang số
def char_to_num(char):
    return ord(char.upper()) - 65

In [3]:
# Chuyển số sang ký tự
def num_to_char(num):
    return chr(int(num) + 65)

In [4]:
# Kiểm tra nếu định thức có nghịch đảo modulo 26
def is_invertible_mod_26(det):
    from math import gcd
    return gcd(det, 26) == 1

In [5]:
# Tính ma trận nghịch đảo modulo 26
def mod_inverse_matrix(matrix, modulus):
    det = int(round(np.linalg.det(matrix))) % modulus
    from math import gcd
    if gcd(det, modulus) != 1:
        raise ValueError(f"Định thức không có nghịch đảo modulo {modulus}. Thử ma trận khóa khác.")
    det_inv = pow(det, -1, modulus)
    adj_matrix = np.round(det * np.linalg.inv(matrix)).astype(int) % modulus
    return (det_inv * adj_matrix) % modulus

In [6]:
def hill_encrypt(plaintext, key_matrix):
    n = len(key_matrix)
    plaintext_numbers = [char_to_num(c) for c in plaintext]
    if len(plaintext_numbers) % n != 0:
        plaintext_numbers.extend([0] * (n - len(plaintext_numbers) % n))  # Thêm padding
    ciphertext = ""
    for i in range(0, len(plaintext_numbers), n):
        chunk = plaintext_numbers[i:i + n]
        result = np.dot(key_matrix, chunk) % 26
        ciphertext += ''.join(num_to_char(num) for num in result)
    return ciphertext

In [7]:
def hill_decrypt(ciphertext, key_matrix):
    n = len(key_matrix)
    key_inverse = mod_inverse_matrix(key_matrix, 26)
    ciphertext_numbers = [char_to_num(c) for c in ciphertext]
    decrypted = ""
    for i in range(0, len(ciphertext_numbers), n):
        chunk = ciphertext_numbers[i:i + n]
        result = np.dot(key_inverse, chunk) % 26
        decrypted += ''.join(num_to_char(num) for num in result)
    return decrypted

In [8]:
def process_file(input_file, key_matrix, mode):
    text = input_file.decode("utf-8").strip().replace(" ", "").upper()
    if mode == "1":
        result = hill_encrypt(text, key_matrix)
        output_file_name = "hill_encrypted.txt"
    else:
        result = hill_decrypt(text, key_matrix)
        output_file_name = "hill_decrypted.txt"
    with open(output_file_name, 'w') as file:
        file.write(result)
    return output_file_name

In [11]:
try:
    mode = input("Chọn chế độ (1: Mã hóa, 2: Giải mã): ")
    key = input("Nhập khóa dạng ma trận vuông (VD: 2x2, nhập '2 3; 1 4'): ")
    key_matrix = np.array([[int(num) for num in row.split()] for row in key.split(";")])
    det = int(round(np.linalg.det(key_matrix))) % 26

    if not is_invertible_mod_26(det):
        raise ValueError(f"Ma trận khóa không hợp lệ: định thức {det} không có nghịch đảo modulo 26.")

    uploaded = files.upload()
    for fn in uploaded.keys():
        processed_file = process_file(uploaded[fn], key_matrix, mode)
        files.download(processed_file)
        print(f"Đã xử lý file {fn}.")
except Exception as e:
    print(f"Lỗi: {e}")

Chọn chế độ (1: Mã hóa, 2: Giải mã): 1
Nhập khóa dạng ma trận vuông (VD: 2x2, nhập '2 3; 1 4'): 2 3; 1 4


Saving test.txt to test (2).txt


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Đã xử lý file test (2).txt.
